[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MouseLand/rastermap/blob/main/notebooks/rastermap_widefield.ipynb)

# Rastermap sorting of widefield neural activity

We will use a widefield imaging recording from mouse cortex from [Musall*, Kaufman* et al 2019](https://doi.org/10.1038/s41593-019-0502-4). The full dataset is available [here](https://labshare.cshl.edu/shares/library/repository/38599/). The imaging was collected while mice performed a decision-making task, the imaging rate was 30 Hz. Instead of single neurons here, each sample is a voxel. We filtered out voxels on the edge of the imaging that had low variance. We will run Rastermap using the principal components of the recording because the full recording is too large to fit in memory.

First we will install the required packages, if not already installed. If on google colab, it will require you to click the "RESTART RUNTIME" button because we are updating numpy.

In [ ]:
!pip install numpy>=1.24 # (required for google colab)
!pip install rastermap 
!pip install matplotlib

### Load data and import libraries

If not already downloaded, the following cell will automatically download the processed data stored [here](https://osf.io/5d8q7).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# importing rastermap
# (this will be slow the first time since it is compiling the numba functions)
from rastermap import Rastermap, utils
from scipy.stats import zscore

# download spontaneous activity
filename = utils.download_data(data_type="widefield")

dat = np.load(filename)

# U0 is voxels by number of components (left singular vectors)
# Vsv is time by number of components (right singular vectors weighted by the singular values)
# (each timepoint is 33 ms)
# sv are the singular values
U0, sv, Vsv = dat["U0"], dat["sv"], dat["Vsv"]
n_voxels, n_time = U0.shape[0], Vsv.shape[0]
print(f"{n_voxels} voxels by {n_time} timepoints")

# XY position of each neuron in the recording
xpos, ypos = dat["xpos"], dat["ypos"]

# load the stimulus times
stims = dat["stims"]
reward_times = dat["reward_times"]
reward_color = [0,0.5,0]
fcolor = np.array([[0,0.5,1], [0,1,1], [1,0,0], 
                [1,0.5,0], [0.8,0.5,0.7]])


# load the behavior and task variables
regressors = dat["regressors"]
behav_idx = dat["behav_idx"] # which regressors are behavioral (not task variables)

### Run Rastermap

Let's sort the single neurons with Rastermap, with clustering and upsampling:

In [ ]:
model = Rastermap(n_clusters=100, # number of clusters to compute
                  n_PCs=U0.shape[1], # number of PCs (precomputed in U0 and Vsv)
                  locality=0.5, # locality in sorting is low here to get more global sorting (this is a value from 0-1)
                  time_lag_window=10, # use future timepoints to compute correlation
                  grid_upsample=10, # default value, 10 is good for large recordings
                  ).fit(Usv = U0 * sv, # left singular vectors weighted by the singular values
                        Vsv = Vsv)     # right singular vectors weighted by the singular values
                                                          
y = model.embedding # neurons x 1
isort = model.isort
Vsv_sub = model.Vsv # these are the PCs across time with the mean across voxels subtracted

Let's create superneurons from Rastermap -- we sort the data and then sum over neighboring voxels:

In [ ]:
nbin = 200 # number of voxels to bin over 
U_sn = utils.bin1d(U0[isort], bin_size=nbin, axis=0) # bin over voxel axis
sn = U_sn @ Vsv_sub.T
sn = zscore(sn, axis=1)

### Visualization

Use the Rastermap sorting to visualize the neural activity (see Figure 4 from the paper for the stimulus legend):

In [ ]:
from matplotlib import patches

def plot_events(ax, stims, reward_times, xmin, xmax):
    """ shade stim times and plot reward times """
    nn = sn.shape[0]
    for k in range(4):
        starts = stims[stims[:,1]==k,0].copy()
        starts = starts[np.logical_and(starts>=xmin, starts<xmax)]
        starts -= xmin
        for n in range(len(starts)):
            start = starts[n]+1
            width = 1.6*30
            # add stimulus patch
            ax.add_patch(
                    patches.Rectangle(xy=(start, 0), width=width,
                            height=nn, facecolor=fcolor[k], 
                            edgecolor=None, alpha=0.2))
            
    for reward_time in reward_times:
        if reward_time >= xmin and reward_time < xmax:
            ax.plot((reward_time-xmin)*np.ones(2), [0, nn],
                        color=reward_color, lw=2)
    ax.set_ylim([0,nn])
    ax.invert_yaxis()

# timepoints to visualize
xmin=7810
xmax=8720

# make figure with grid for easy plotting
fig = plt.figure(figsize=(12,6), dpi=200)
grid = plt.GridSpec(9, 20, figure=fig, wspace = 0.05, hspace = 0.3)

# plot superneuron activity
ax = plt.subplot(grid[1:, :-1])
ax.imshow(sn[:, xmin:xmax], cmap="gray_r", vmin=0, vmax=1.1, aspect="auto")
ax.set_xlabel("time")
ax.set_ylabel("superneurons")
plot_events(ax, stims, reward_times, xmin, xmax)

ax = plt.subplot(grid[1:, -1])
ax.imshow(np.arange(0, len(sn))[:,np.newaxis], cmap="gist_ncar", aspect="auto")
ax.axis("off")


Color the voxels by their position in the rastermap:

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(xpos, ypos, s=1, c=y, cmap="gist_ncar", alpha=0.25)
plt.xlabel('X position')
plt.ylabel('Y position')
plt.axis("square");

### Neural activity prediction from behavior

We can use ridge regression to predict superneuron activity from the task variables and the behavioral variables. We will first install the code package with the ridge regression and helper functions implemented:

In [ ]:
!pip install neuropop

Predict from both task and behavioral variables, and predict from behavioral variables alone:

In [ ]:
from neuropop import linear_prediction
import torch

ve, _, sn_pred, itest = linear_prediction.prediction_wrapper(regressors, sn.T, lam=1e4, 
                                                             device=torch.device("cpu"))
sn_pred = sn_pred.T
itest = itest.flatten()
print(f"prediction from task and behavioral variables: \t{ve:.3f}")

ve, _, sn_pred_beh, itest = linear_prediction.prediction_wrapper(regressors[:,behav_idx], 
                                                                 sn.T, lam=1e4,
                                                                 device=torch.device("cpu"))
itest = itest.flatten()
sn_pred_beh = sn_pred_beh.T
print(f"prediction from only behavioral variables: \t{ve:.3f}")


Visualize the predictions on test data:

In [ ]:
# timepoints to visualize
xmin=820
xmax=1730

# make figure with grid for easy plotting
fig = plt.figure(figsize=(10,14), dpi=200)
grid = plt.GridSpec(4, 1, figure=fig, wspace = 0.05, hspace = 0.3)

titles = ["(i) widefield imaging during a decision-making task",
                "(ii) prediction of activity from task and behavior variables",
                "(iii) prediction of activity from behavior variables only",
                "(iv) difference between (ii) and (iii)"]

for j in range(4):
    # plot superneuron activity
    ax = plt.subplot(grid[j])
    if j==0:
        sp = sn[:, itest[xmin:xmax]]
    elif j==1:
        sp = sn_pred[:, xmin:xmax]
    elif j==2:
        sp = sn_pred_beh[:, xmin:xmax]
    else:
        sp = sn_pred[:, xmin:xmax] - sn_pred_beh[:, xmin:xmax]

    ax.imshow(sp, cmap="gray_r" if j<3 else "RdBu_r", 
                vmin=-1.1*(j==3), vmax=1.1, aspect="auto")
    if j==3:
        ax.set_xlabel("time")
    ax.set_ylabel("superneurons")
    ax.set_title(titles[j])
    if j<3:
        plot_events(ax, stims, reward_times, itest[xmin], itest[xmax])


### Settings

You can see all the rastermap settings with `Rastermap?`

In [ ]:
Rastermap?

### Outputs

All the attributes assigned to the Rastermap `model` are listed with `Rastermap.fit?`

In [ ]:
Rastermap.fit?